In [0]:
raw_path = "assertbundles.bronze.raw_employee_data"
raw_df = spark.read.table(raw_path)
raw_df.show(truncate=False)


In [0]:
from pyspark.sql.functions import col, trim, lower
from pyspark.sql.types import IntegerType, FloatType

# 1. Drop rows with nulls in essential columns (id, age, department, email)
df_clean = raw_df.dropna(subset=["id", "age", "department", "email"])

# 2. Filter only rows where 'age' is numeric
df_clean = df_clean.filter(col("age").rlike("^[0-9]+$"))

# 3. Trim spaces and normalize string columns (name, department, email)
df_clean = df_clean.withColumn("name", trim(col("name"))) \
                   .withColumn("department", trim(col("department"))) \
                   .withColumn("email", trim(lower(col("email"))))

# 4. Validate email format with regex
email_regex = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
df_clean = df_clean.filter(col("email").rlike(email_regex))

# 5. Cast age and salary to correct data types
df_clean = df_clean.withColumn("age", col("age").cast(IntegerType())) \
                   .withColumn("salary", col("salary").cast(FloatType()))

# 6. Write cleaned data into managed Spark table (overwrite if exists)
df_clean.write.mode("overwrite").saveAsTable("assertbundles.silver.clean_employee_data")

# Optional: Show the cleaned data
df_clean.show(truncate=False)
